In [1]:
from numpy import *
import astropy.units as u
from astropy.constants import G, M_jup, R_jup, M_earth, R_earth, L_sun, M_sun

[Rigliaco et al. 2012](http://adsabs.harvard.edu/abs/2012A%26A...548A..56R)

In [2]:
def line_lum(dist,zeropt,filtwid,star_mag,contr):
    """
    Given distance, flux zeropoint, stellar R-band magnitude, and H-alpha contrast (or contrast limit),
    calculate line luminosity in Watts. 
    """
    delta_mag = -2.5*log10(contr)
    L_line = 4*pi*dist**2*zeropt*filtwid*10**((star_mag+delta_mag)/-2.5)
    L_line = L_line.decompose().to(u.W)
    return(L_line)

#next two are empirical T-Tauri relationships from Rigliaco 2012

def acc_lum(L_line):
    """
    Translate H-alpha line luminosity to accretion luminosity. From empirical 
    T-Tauri relationships in Rigliaco et al. 2102
    """
    b = 2.27
    a = 1.25
    log_acc = b+a*log10(L_line/L_sun)
    L_acc=10**log_acc*L_sun
    return(L_acc)

def acc_rate(L_acc, R, M):
    """
    Translate an accretion luminosity and planet mass/radius to accretion rate in jupiter masses per year.
    Also following Rigliaco et al. 2012.
    """
    mdot = 1.25*L_acc*R/(G*M)
    mdot = mdot.decompose().to(u.Mjup/u.yr)
    return(mdot)

In [3]:
## check HD142527 calculation from Close et al. 2014

mag = 11.6-0.57
contr = 1e-2
dist = 145*u.pc
zeropt = 2.339e-5*u.erg/u.cm**2/u.s/u.um
L_line = line_lum(dist,zeropt,0.006*u.um,mag,contr)
log10(L_line/L_sun)
L_acc=acc_lum(L_line)
mdot=acc_rate(L_acc,1.55*R_jup,2*M_jup)
mdot

#interestingly, it appears that Laird used the incorrect L_sun 
#(he used 3.28e33, but it should have been 3.85e33). Otherwise, same answer

<Quantity 1.78873585e-06 jupiterMass / yr>

In [4]:
L_line

<Quantity 1.36718861e+22 W>

In [17]:
import numpy as np
Acc_lum = 10**(-2.9)*L_sun
Mdot = acc_rate(Acc_lum,4.6*R_jup,31*M_jup)
Mnew = Mdot.to(u.Msun/u.yr)
np.log10(Mnew.value)

-9.096611172975074

In [5]:
## Zhou 2014 results
GSC_logL = -5.03
GSC_Lacc=acc_lum(10**GSC_logL*L_sun)
GSC_Mdot = acc_rate(GSC_Lacc,1.8*R_jup,15*M_jup)
print(L_line)
print(log10(GSC_Mdot.value))

GQLup_logL = -4.69
GQLup_Lacc=acc_lum(10**GQLup_logL*L_sun)
GQLup_Mdot = acc_rate(GQLup_Lacc,4.6*R_jup,31*M_jup)
print(log10(GQLup_Mdot.value))

DHTau_logL = -6.19
DHTau_Lacc=acc_lum(10**DHTau_logL*L_sun)
DHTau_Mdot = acc_rate(DHTau_Lacc,2.7*R_jup,11*M_jup)
print(log10(DHTau_Mdot.value))

1.3671886083836084e+22 W
-7.286144871546158
-6.768929979746482
-8.425355038593022


In [6]:
#sanity check of line luminosity
GSC_Hamag = 15.7
GSC_AV = 0.2
GSC_mag = GSC_Hamag-GSC_AV
GSC_contr =  7.33e-18/7.08e-16 #from Table 1 cont/Ha flux
GSC_dist = 145*u.pc
filterwid = 18*u.AA
zeropt = 1.434e-9*u.erg/u.cm**2/u.s/u.AA #19.863 mag

GSC_L_line = line_lum(GSC_dist,zeropt,filterwid,GSC_mag,GSC_contr)
GSC_Lacc=acc_lum(GSC_L_line)
GSC_Mdot = acc_rate(GSC_Lacc,1.8*R_jup,15*M_jup)
print(log10(GSC_Mdot.value))

GQLup_Hamag = 15.9
GQLup_AV = 1.5
GQLup_mag = GQLup_Hamag-GQLup_AV
GQLup_contr = 5.03e-17/5.92e-16
GQLup_dist = 155*u.pc
GQLup_L_line = line_lum(GQLup_dist,zeropt,filterwid,GQLup_mag,GQLup_contr)
GQLup_Lacc=acc_lum(GQLup_L_line)
GQLup_Mdot = acc_rate(GQLup_Lacc,4.6*R_jup,31*M_jup)
print(log10(GQLup_Mdot.value))

DHTau_Hamag = 19
DHTau_AV = 0.7
DHTau_mag = DHTau_Hamag-DHTau_AV
DHTau_contr = 1.35e-18/3.48e-17
DHTau_dist = 145*u.pc
DHTau_L_line = line_lum(DHTau_dist,zeropt,filterwid,DHTau_mag,DHTau_contr)
DHTau_Lacc=acc_lum(DHTau_L_line)
DHTau_Mdot = acc_rate(DHTau_Lacc,2.7*R_jup,11*M_jup)
print(log10(DHTau_Mdot.value))

-9.69293561753221
-7.835592034167181
-10.065041025082742


## Potential WFIRST Filter Widths

**Filters:**
Narrowband (Hubble WFC3 F656N) 14A = 0.0014 micron
Wider (Hubble WFC3 F657N incl. [NIII]) 94A ~ 0.01 micron

**R~50 IFS**
Bandpass = 600-970nm —>  width ~ 8nm = 0.008


In [14]:
#Accreting Earth in HL Tau at contrast threshhold 1e-5

zeropt = 2.25e-5*u.erg/u.cm**2/u.s/u.um #R band Johnson zeropoint
dist = 140*u.pc
mag = 14.21
filterwid = 0.01*u.um
contrast = 1e-5

L_line = line_lum(dist,zeropt,filterwid,mag,contrast)
L_acc = acc_lum(L_line)
mdot=acc_rate(L_acc,R_earth,M_earth)
mdot.to(u.Mearth/u.yr)

<Quantity 1.57114114e-07 earthMass / yr>

<Quantity 1.57114114e-07 earthMass / yr>